루브릭
1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.  
구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.  
2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.  
seq2seq 모델 훈련결과를 그래프로 출력해보고, validation loss그래프가 우하향하는 경향성을 보이며 학습이 진행됨이 확인되었다.  
3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.  
테스트용 디코더 모델이 정상적으로 만들어졌으며, input(영어)와 output(프랑스어) 모두 한글로 번역해서 결과를 출력해보았고, 둘의 내용이 유사함을 확인하였다.  

# 단어 Level 번역기 만들기
데이터는 상위 33,000개의 샘플만 사용  
1.정제, 정규화, 전처리  
구두점 분리, 소문자 전환, 띄어쓰기 단위 토큰화  
2.디코더 문장에 시작 및 종료 토큰 삽입  
3.케라스 토크나이저로 텍스트 숫자 변환  
4.임배딩 레이어 사용하기(노드 10-10 step 4 참조  
5.모델 구현하기  
sparse categorical entropy loss사용  
6.모델 평가하기  


In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
# lines.sample(5) #샘플 5개 출력
lines.head(5)

전체 샘플의 수 : 197463


,eng,fra,cc
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [3]:
# 3만 3천개의 데이터만 사용한다.(train 30000, test 3000)
lines = lines[['eng', 'fra']][:33000]
lines.head()

,eng,fra
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [4]:
# 시작 토큰과 종료 토큰 추가
sos_token = '<sos>'
eos_token = '<eos>'
lines.fra = lines.fra.apply(lambda x : sos_token + x + eos_token)
print('전체 샘플의 수 :',len(lines))
lines.head(10)

전체 샘플의 수 : 33000


,eng,fra
0,Go.,<sos>Va !<eos>
1,Go.,<sos>Marche.<eos>
2,Go.,<sos>En route !<eos>
3,Go.,<sos>Bouge !<eos>
4,Hi.,<sos>Salut !<eos>
5,Hi.,<sos>Salut.<eos>
6,Run!,<sos>Cours !<eos>
7,Run!,<sos>Courez !<eos>
8,Run!,<sos>Prenez vos jambes à vos cous !<eos>
9,Run!,<sos>File !<eos>


In [5]:
# 모든 대문자 소문자 변환
lines['eng']=lines['eng'].str.lower()
lines['fra']=lines['fra'].str.lower()
lines.head(10)

,eng,fra
0,go.,<sos>va !<eos>
1,go.,<sos>marche.<eos>
2,go.,<sos>en route !<eos>
3,go.,<sos>bouge !<eos>
4,hi.,<sos>salut !<eos>
5,hi.,<sos>salut.<eos>
6,run!,<sos>cours !<eos>
7,run!,<sos>courez !<eos>
8,run!,<sos>prenez vos jambes à vos cous !<eos>
9,run!,<sos>file !<eos>


In [6]:
# 영어 토큰화
eng_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3] # 0,1,2모두 go 기 때문에 같을 수 밖에 없으니 당황하지 말 것

[[22], [22], [22]]

In [7]:
# 프랑스어 토큰화
fra_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 59, 2], [1, 351, 2], [1, 22, 499, 2]]

In [8]:
# 디코더 토큰 중 디코더 입력 데이터에 종료 토큰과 디코더 target 데이터에 시작 토큰 제거
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]
decoder_target

KeyError: '<eos>'

In [14]:
dir(fra_tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_keras_api_names',
 '_keras_api_names_v1',
 'char_level',
 'document_count',
 'filters',
 'fit_on_sequences',
 'fit_on_texts',
 'get_config',
 'index_docs',
 'index_word',
 'lower',
 'num_words',
 'oov_token',
 'sequences_to_matrix',
 'sequences_to_texts',
 'sequences_to_texts_generator',
 'split',
 'texts_to_matrix',
 'texts_to_sequences',
 'texts_to_sequences_generator',
 'to_json',
 'word_counts',
 'word_docs',
 'word_index']

In [12]:
print(target_text.index)

<built-in method index of list object at 0x7f49d9107440>
